# Importing Libraries

In [ ]:
!pip install keras_tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.4 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive')

#Importing dataset

In [4]:
dataframe = pd.read_excel('chiefs_knife_dataset.xlsx')
index_Ra = dataframe.columns.get_loc('Ra') # index of the surface roughness column for inserting the class. label

lower_specification_limit = 0.125 # lower bound of good quality product region
upper_specification_limit = 0.215  # upper bound of good quality product region

good_product_range = (dataframe['Ra']>=lower_specification_limit) & (dataframe['Ra'] < upper_specification_limit)
dataframe.insert(index_Ra +1, 'Good_Quality',good_product_range.astype(int))

In [6]:
dataframe

,Number,Name,Linie,Ra_ganz,Rq_ganz,Rz_ganz,Rt_ganz,Ra_Messlinienlange,Rq_Messlinienlange,Rz_Messlinienlange,...,DFT_Median_sobel_ST_Linienbreite,DFT_Median_sobel_Max,DFT_Median_sobel_Min,DFT_Median_sobel_Bereich,Ra,Good_Quality,Rz,Rq,Rt,Gloss
0,0,Koch-A0-T0-01_cropped_Messpunkt,0,6.203955,7.784391,38.901961,47.843137,6.527037,8.291151,36.470588,...,3.893554,26,2,24,0.16,1,1.19,0.21,1.45,139.16
1,1,Koch-A0-T0-01_cropped_Messpunkt,1,6.308510,7.861995,38.823529,48.235294,6.817339,8.483026,36.627451,...,4.275049,26,4,22,0.16,1,1.19,0.21,1.45,139.16
2,2,Koch-A0-T0-01_cropped_Messpunkt,2,6.113172,7.635026,40.784314,47.450980,6.711293,8.389655,35.921569,...,2.732972,16,5,11,0.16,1,1.19,0.21,1.45,139.16
3,3,Koch-A0-T0-01_cropped_Messpunkt,3,6.025495,7.531023,37.333333,44.705882,6.279870,8.056241,36.078431,...,3.011902,27,5,22,0.16,1,1.19,0.21,1.45,139.16
4,4,Koch-A0-T0-01_cropped_Messpunkt,4,6.559739,8.139329,36.941176,44.313725,7.070941,8.714572,37.411765,...,3.430747,28,1,27,0.16,1,1.19,0.21,1.45,139.16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8505,8505,Koch-VVS7-20_cropped_Messpunkt,5,4.301919,5.408163,28.470588,33.333333,4.333209,5.424723,25.725490,...,4.246271,23,2,21,0.14,1,0.92,0.19,1.54,154.08
8506,8506,Koch-VVS7-20_cropped_Messpunkt,6,4.267380,5.309656,26.901961,31.764706,4.435460,5.412355,22.901961,...,3.592141,21,2,19,0.14,1,0.92,0.19,1.54,154.08
8507,8507,Koch-VVS7-20_cropped_Messpunkt,7,4.320298,5.371672,29.019608,31.764706,4.434845,5.449217,23.215686,...,3.607983,27,2,25,0.14,1,0.92,0.19,1.54,154.08
8508,8508,Koch-VVS7-20_cropped_Messpunkt,8,4.413511,5.401825,27.529412,31.372549,4.160347,5.081846,23.215686,...,4.630190,25,3,22,0.14,1,0.92,0.19,1.54,154.08


In [ ]:
features = dataframe.loc[:,'Original_Linienanzahl':'DFT_Median_sobel_Bereich'].values
target_regression = dataframe['Ra'].values
target_classification = dataframe['Good_Quality'].values

In [ ]:
one_prob = (np.count_nonzero(target_classification == 1))/len(target_classification)
zero_prob = (np.count_nonzero(target_classification == 0))/len(target_classification)
print(f'{one_prob=}')
print(f'{zero_prob=}')
print(np.count_nonzero(target_classification == 1))
print(np.count_nonzero(target_classification == 0))
print(len(target_classification))


one_prob=0.5264394829612221
zero_prob=0.4735605170387779
4480
4030
8510


In [ ]:
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [ ]:
test_size = 0.2
validation_size = 0.1

In [ ]:
sc = MinMaxScaler()
features = sc.fit_transform(features)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target_classification, test_size=test_size,shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=validation_size,shuffle=True, random_state=0)

#Hyperparameters definition

In [ ]:
import keras_tuner as kt
from keras_tuner import HyperParameters

neurons_max = 1024
neurons_min = 32
neurons_step = 16
dropout_min = 0.0
dropout_max = 0.5
dropout_step = 0.1
l2_min = 0.0
l2_max = 0.1
l2_step = 0.01
learning_rates = [1e-2, 1e-3, 1e-4,1e-5]
activations = ['relu', 'tanh']

hp = kt.HyperParameters()

#Model Definition

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_model(hp):
  """
  create a NN architecture for the classifiction task
  hp: hyperparameter for hyperparameter tuning
  """
  model = Sequential()

  model.add(Dense(units = hp.Int('hidden_1', min_value=neurons_min, max_value=neurons_max, step=neurons_step),
                  activation =hp.Choice('activation_1', values=activations),
                  kernel_regularizer=l2(hp.Float('l2_1', min_value=l2_min, max_value=l2_max, step=l2_step)),
                  input_dim=X_train.shape[1]))
  model.add(BatchNormalization())
  model.add(Dropout(rate=hp.Float('dropout_1', min_value=dropout_min, max_value=dropout_max, step=dropout_step)))

  model.add(Dense(units = hp.Int('hidden_2', min_value=neurons_min, max_value=neurons_max, step=neurons_step),
                  activation =hp.Choice('activation_2', values=activations),
                  kernel_regularizer=l2(hp.Float('l2_2', min_value=l2_min, max_value=l2_max, step=l2_step))))
  model.add(BatchNormalization())
  model.add(Dropout(rate=hp.Float('dropout_2', min_value=dropout_min, max_value=dropout_max, step=dropout_step)))

  model.add(Dense(units = hp.Int('hidden_3', min_value=neurons_min, max_value=neurons_max, step=neurons_step),
                  activation =hp.Choice('activation_3', values=activations),
                  kernel_regularizer=l2(hp.Float('l2_3', min_value=l2_min, max_value=l2_max, step=l2_step))))
  model.add(BatchNormalization())
  model.add(Dropout(rate=hp.Float('dropout_3', min_value=dropout_min, max_value=dropout_max, step=dropout_step)))

  model.add(Dense(units = hp.Int('hidden_4', min_value=neurons_min, max_value=neurons_max, step=neurons_step),
                  activation =hp.Choice('activation_4', values=activations),
                  kernel_regularizer=l2(hp.Float('l2_4', min_value=l2_min, max_value=l2_max, step=l2_step))))
  model.add(BatchNormalization())
  model.add(Dropout(rate=hp.Float('dropout_4', min_value=dropout_min, max_value=dropout_max, step=dropout_step)))

  model.add(Dense(units= 1, activation ='sigmoid'))
  model.compile(optimizer = Adam(hp.Choice('learning_rate', values=learning_rates)),loss='binary_crossentropy',metrics = ['accuracy'])
  return model

In [ ]:
# build_model(hp).summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 32)                992       
                                                                 
 batch_normalization_4 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 batch_normalization_5 (Bat  (None, 32)                128       
 chNormalization)                                                
                                                                 
 dropout_5 (Dropout)         (None, 32)               

In [ ]:
tuner = kt.RandomSearch(
hypermodel=build_model,
objective="val_accuracy",
max_trials=60,
executions_per_trial=1,
overwrite=True,
directory="hyperparameter_tuning",
project_name="simple",
)

tuner.search_space_summary()

Search space summary
Default search space size: 17
hidden_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 16, 'sampling': 'linear'}
activation_1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
l2_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.1, 'step': 0.01, 'sampling': 'linear'}
dropout_1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
hidden_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 16, 'sampling': 'linear'}
activation_2 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
l2_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.1, 'step': 0.01, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
hidden_

In [ ]:
tuner.search(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Trial 60 Complete [00h 01m 54s]
val_accuracy: 0.5682819485664368

Best val_accuracy So Far: 0.7063142657279968
Total elapsed time: 01h 18m 18s


In [ ]:
models = tuner.get_best_models(num_models=3)
models[0].summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 288)               8928      
                                                                 
 dropout (Dropout)           (None, 288)               0         
                                                                 
 dense_1 (Dense)             (None, 432)               124848    
                                                                 
 dropout_1 (Dropout)         (None, 432)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                27712     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 240)               1

In [ ]:
tuner.results_summary()

Results summary
Results in hyperparameter_tuning/simple
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 10 summary
Hyperparameters:
hidden_1: 96
activation_1: tanh
l2_1: 0.02
dropout_1: 0.1
hidden_2: 992
activation_2: relu
l2_2: 0.07
dropout_2: 0.1
hidden_3: 672
activation_3: relu
l2_3: 0.04
dropout_3: 0.2
hidden_4: 432
activation_4: relu
l2_4: 0.02
dropout_4: 0.0
learning_rate: 1e-05
Score: 0.7063142657279968

Trial 44 summary
Hyperparameters:
hidden_1: 544
activation_1: relu
l2_1: 0.08
dropout_1: 0.1
hidden_2: 832
activation_2: tanh
l2_2: 0.02
dropout_2: 0.2
hidden_3: 368
activation_3: relu
l2_3: 0.1
dropout_3: 0.2
hidden_4: 176
activation_4: relu
l2_4: 0.07
dropout_4: 0.30000000000000004
learning_rate: 1e-05
Score: 0.691629946231842

Trial 26 summary
Hyperparameters:
hidden_1: 256
activation_1: tanh
l2_1: 0.07
dropout_1: 0.0
hidden_2: 352
activation_2: relu
l2_2: 0.09
dropout_2: 0.4
hidden_3: 640
activation_3: relu
l2_3: 0.08
dropout_3: 0.1
hidden_4: 30

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'hidden_1': 96,
 'activation_1': 'tanh',
 'l2_1': 0.02,
 'dropout_1': 0.1,
 'hidden_2': 992,
 'activation_2': 'relu',
 'l2_2': 0.07,
 'dropout_2': 0.1,
 'hidden_3': 672,
 'activation_3': 'relu',
 'l2_3': 0.04,
 'dropout_3': 0.2,
 'hidden_4': 432,
 'activation_4': 'relu',
 'l2_4': 0.02,
 'dropout_4': 0.0,
 'learning_rate': 1e-05}

In [ ]:
# models[0].fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# models[0].evaluate(X_test, y_test)

In [ ]:
# best_hp = tuner.get_best_hyperparameters(3)
# [best_hp[i].values for i in range(len(best_hp))]

In [ ]:
para = {'hidden_1': 96,
 'activation_1': 'tanh',
 'l2_1': 0.02,
 'dropout_1': 0.1,
 'hidden_2': 992,
 'activation_2': 'relu',
 'l2_2': 0.07,
 'dropout_2': 0.1,
 'hidden_3': 672,
 'activation_3': 'relu',
 'l2_3': 0.04,
 'dropout_3': 0.2,
 'hidden_4': 432,
 'activation_4': 'relu',
 'l2_4': 0.02,
 'dropout_4': 0.0,
 'learning_rate': 1e-05}

In [ ]:
def ann(**best_hp):
  """
  create a NN architecture for the classifiction task using the best hyperparameters
  hp: hyperparameter for hyperparameter tuning
  """
  model_best = Sequential()

  model_best.add(Dense(units = best_hp['hidden_1'],
                  activation =best_hp['activation_1'],
                  kernel_regularizer=l2(best_hp['l2_1']),
                  input_dim=X_train.shape[1]))

  model_best.add(Dropout(rate=best_hp['dropout_1']))

  model_best.add(Dense(units = best_hp['hidden_2'],
                  activation =best_hp['activation_2'],
                  kernel_regularizer=l2(best_hp['l2_2'])))

  model_best.add(Dropout(rate=best_hp['dropout_2']))

  model_best.add(Dense(units = best_hp['hidden_3'],
                  activation =best_hp['activation_3'],
                  kernel_regularizer=l2(best_hp['l2_3'])))

  model_best.add(Dropout(rate=best_hp['dropout_3']))

  model_best.add(Dense(units = best_hp['hidden_4'],
                  activation =best_hp['activation_4'],
                  kernel_regularizer=l2(best_hp['l2_4'])))

  model_best.add(Dropout(rate=best_hp['dropout_4']))

  model_best.add(Dense(units= 1, activation ='sigmoid'))
  model_best.compile(optimizer = Adam(learning_rate=best_hp['learning_rate']),loss='binary_crossentropy',metrics = ['accuracy'])
  history = model_best.fit(X_train, y_train, batch_size=8,epochs=100, validation_data=(X_val, y_val))
  return model_best, history

In [ ]:
model, history = ann(**para)

Epoch 1/100
766/766 [==============================] - 15s 18ms/step - loss: 47.7926 - accuracy: 0.5285 - val_loss: 40.0129 - val_accuracy: 0.5213
Epoch 2/100
766/766 [==============================] - 13s 17ms/step - loss: 33.7128 - accuracy: 0.5285 - val_loss: 28.0120 - val_accuracy: 0.5213
Epoch 3/100
766/766 [==============================] - 13s 18ms/step - loss: 23.3897 - accuracy: 0.5283 - val_loss: 19.2150 - val_accuracy: 0.5213
Epoch 4/100
766/766 [==============================] - 13s 17ms/step - loss: 15.8569 - accuracy: 0.5283 - val_loss: 12.8428 - val_accuracy: 0.5213
Epoch 5/100
766/766 [==============================] - 13s 17ms/step - loss: 10.4581 - accuracy: 0.5283 - val_loss: 8.3401 - val_accuracy: 0.5213
Epoch 6/100
766/766 [==============================] - 13s 17ms/step - loss: 6.7065 - accuracy: 0.5283 - val_loss: 5.2779 - val_accuracy: 0.5213
Epoch 7/100
766/766 [==============================] - 13s 17ms/step - loss: 4.2142 - accuracy: 0.5283 - val_loss: 3.3033

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
matrix_conf=confusion_matrix(y_test, model.predict(X_test).round())
accuracy_model=accuracy_score(y_test, model.predict(X_test).round())
print(f"{matrix_conf=}")
print(f'{accuracy_model=}')

NameError: name 'models' is not defined

In [ ]:
# import matplotlib.pyplot as plt
# def show_loss_acc(h):
#   plt.figure(figsize=(12, 8))
#   plt.subplot(2, 2, 1)
#   plt.plot(h['loss'], label='train loss')
#   plt.plot(h['val_loss'], label='val loss')
#   plt.legend()
#   plt.subplot(2, 2, 2)
#   plt.plot(h['accuracy'], label='train accuracy')
#   plt.plot(h['val_accuracy'], label='val accuracy')
#   plt.legend()
#   plt.show()
# show_loss_acc(history.history)

In [ ]:
# from keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import cross_val_score

In [ ]:
# # Function to create the KerasClassifier
# def create_model():
#     from kerastuner import HyperParameters
#     hp = HyperParameters()
#     return build_model(hp)

In [ ]:
# # Wrap the model using KerasClassifier
# model = KerasClassifier(build_fn=create_model, epochs=50, batch_size=10, verbose=0)

In [ ]:
# # Define the number of folds for cross-validation
# n_folds = 5

In [ ]:
# # Perform cross-validation
# scores = cross_val_score(model, X_train, y_train, cv=n_folds, scoring='accuracy')

In [ ]:
# # Print the cross-validation scores
# print(f'Cross-validation scores: {scores}')
# print(f'Mean cross-validation score: {np.mean(scores)}')
# print(f'Standard deviation of cross-validation score: {np.std(scores)}')